In [66]:
import os
import sys
sys.path.append('../..')
from consecution import Node, Pipeline, GlobalState
import random
import pandas as pd
import numpy as np
import scipy as scp
from scipy import stats
from collections import namedtuple
import datetime
import copy
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import seaborn as sns
import pylab as pl
import random
%matplotlib inline


In [83]:
starting = parse('1/1/2016')
n_days = 365
one_day = datetime.timedelta(days=1)
days = [starting + nn * one_day for nn in range(n_days)]
baseline_recs_per_day = 100
baseline_mu = 10
sigma = 100

def get_day_fractions(t):
    return [.2, .3, .4, .5, .6, .7, .1][t.weekday()]

def get_hour_fraction(t):
    today = datetime.datetime(t.year, t.month, t.day)
    tomorrow = today + datetime.timedelta(days=1)
    fraction = (t - today).total_seconds() / float((tomorrow - today).total_seconds())
    return fraction
    


In [88]:
rec_list = []
for day in days:
    num_recs = int(baseline_recs_per_day * get_day_fractions(day))
    for nr in range(num_recs):
        num_seconds = random.randint(3600*8, 3600*(17) - 1)
        time_stamp = day + datetime.timedelta(seconds=num_seconds)
        mu = baseline_mu * 10*get_hour_fraction(time_stamp)
        value = 1 *mu + sigma * np.random.randn()
        rec_list.append((time_stamp, value))
df = pd.DataFrame(rec_list, columns=['time', 'amount'])
#df = df.sort_values(by='time')
df = df.set_index('time').sort_index()
df.head()

,amount
time,
2016-01-01 08:05:59,61.718588
2016-01-01 08:10:31,48.819577
2016-01-01 08:22:12,122.051658
2016-01-01 08:38:15,-33.488257
2016-01-01 08:41:20,63.819369


In [89]:
os.system('rm cash_flow*.csv')
for file_name, group in df.groupby(by=lambda ind: 'cash_flows_{}-{:02d}.csv'.format(ind.year, ind.month)):
    group.reset_index().to_csv(file_name, index=False)
